# モデルを考察する（１）

モデルの振る舞いを観察し、その動作原理を考察します。


### お決まりの準備作業

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import biosim_course

## モデルの構造を参考に、シミュレーション結果の意味を考える

### 状態遷移図

* 下図は、論文を参考に、モデル内の反応をまとめた **状態遷移図** です。
* 着色されている文字は、モデル中での `Variable` 名です。
* 白抜きの番号は反応の番号で、数字は、`Process` 名 `Rn` の _n_ に相当します。

![Model](./Lec-11-Tyson1991.png)

* `CT` は cdc2 の総量（`C2` + `CP` + `pM` + `M`） です。
    * cdc2 については、モデル中に合成・分解は表現されていません。つまり、`CT` は一定（定数）です。
    * すなわちこのモデルでは、cdc2 はつねに一定量存在するように、合成・分解が調整されていると仮定しているといえます。
* `YT` は cyclin の総量（`Y` + `YP` + `pM` + `M`） です。


### 微分方程式

* 以下に、各反応の微分方程式を示します。反応４以外は、マルサスの人口モデルと同じ考え方に基づく質量作用則（Mass Action）で表現されています。
* 反応４に含まれる関数 _f_ の内容を、最下段に示しています。
* 反応４には、`M` によるポジティブフィードバックが表現されています。

![Model](./Lec-11-Eqns.png)

* _k_<sub>_1_</sub> 〜 _k_<sub>_9_</sub>, _k'_<sub>_4_</sub> は速度定数で、実験による計測などで決まります。


### `Tyson1991.em` で用いられているパラメータの値

* 具体的なパラメータの値は下の表の通りです。

![Parameters](./Lec-11-Param.png)

* _k_<sub>_2_</sub>, _k_<sub>_5_</sub> をゼロとみなしています。
* _k_<sub>_8_</sub>[P] >> _k_<sub>_9_</sub> >> _k_<sub>_6_</sub> としています。
    * すなわち、cdc2 の合成・分解に関わる反応では、`M` が分解する反応（`R6`）よりも、単量体の cdc2 のリン酸化（`R8`）・脱リン酸化（`R9`）の方が圧倒的に速いと仮定しています。
    * cdc2 のリン酸化（`R8`）・脱リン酸化（`R9`）では、リン酸化の方が圧倒的に速いと仮定しています。
    * これらを総合すると、`M` が分解してできた cdc2（`C2`） は、高速にリン酸化されて `CP` になり、`C2` の状態で存在している cdc2 はほとんど存在しないことを仮定しているといえます。

* _k_<sub>_4_</sub>, _k_<sub>_6_</sub> は、カッコ内の数値の範囲内で変更可能なパラメータです。


In [ ]:
EM = '''
Stepper ODEStepper( ODE ) {}
Stepper PassiveStepper( PSV ) {}

System System( / )
{
    StepperID  ODE;
    Name  default;
}

System System( /Cell )
{
    StepperID  ODE;

    Variable Variable( SIZE )
    {
        Value  1.0;
    }
    
    Variable Variable( C2 )
    {
        Name  cdc2k;
        Value  0.0;
    }
    
    Variable Variable( CP )
    {
        Name  "cdc2k-P";
        Value  0.75;
    }
    
    Variable Variable( M )
    {
        Name  "p-cyclin_cdc2";
        Value  0.0;
    }
    
    Variable Variable( pM )
    {
        Name  "p-cyclin_cdc2-p";
        Value  0.25;
    }
    
    Variable Variable( Y )
    {
        Name  cyclin;
        Value  0.0;
    }
    
    Variable Variable( YP )
    {
        Name  "p-cyclin";
        Value  0.0;
    }
    
    Variable Variable( YT )
    {
        Name  total_cyclin;
        Value  0.25;
    }
    
    Variable Variable( CT )
    {
        Name  total_cdc2;
        Value  1.0;
    }

### Reactions ###

    Process ExpressionFluxProcess( R1 )
    {
        Name  "cyclin biosynthesis";
        k1aa  0.015;
        Expression  "k1aa / CT.Value";
        VariableReferenceList
            [ Y    :.:Y   1 ]
            [ CT   :.:CT  0 ];
    }
    
    Process ExpressionFluxProcess( R2 )
    {
        Name  "default degradation of cyclin";
        k2  0.0;
        Expression  "k2 * Y.Value";
        VariableReferenceList
            [ Y   :.:Y  -1 ];
    }
    
    Process ExpressionFluxProcess( R3 )
    {
        Name  "cyclin cdc2k-p association";
        k3  200.0;
        Expression  "k3 * CP.Value * Y.Value / CT.Value";
        VariableReferenceList
            [ CP   :.:CP  -1 ]
            [ Y    :.:Y   -1 ]
            [ pM   :.:pM   1 ]
            [ CT   :.:CT   0 ];
    }
    
    Process ExpressionFluxProcess( R4 )
    {
        Name  "activation of cdc2 kinase";
        k4  180.0;
        k4prime  0.018;
        Expression  "pM.Value * (k4prime + k4 * pow(M.Value / CT.Value, 2))";
        VariableReferenceList
            [ pM   :.:pM  -1 ]
            [ M    :.:M    1 ]
            [ CT   :.:CT   0 ];
    }
    
    Process ExpressionFluxProcess( R5 )
    {
        Name  "deactivation of cdc2 kinase";
        k5_P  0.0;
        Expression  "k5_P * M.Value";
        VariableReferenceList
            [ M    :.:M   -1 ]
            [ pM   :.:pM   1 ];
    }
    
    Process ExpressionFluxProcess( R6 )
    {
        Name  "cyclin_cdc2k dissociation";
        k6  1.0;
        Expression  "k6 * M.Value";
        VariableReferenceList
            [ M    :.:M   -1 ]
            [ C2   :.:C2   1 ]
            [ YP   :.:YP   1 ];
    }
    
    Process ExpressionFluxProcess( R7 )
    {
        Name  "cdc2 kinase triggered degration of cyclin";
        k7  0.6;
        Expression  "k7 * YP.Value";
        VariableReferenceList
            [ YP   :.:YP  -1 ];
    }
    
    Process ExpressionFluxProcess( R8 )
    {
        Name  "cdc2k phosphorylation";
        k8_P  1000000.0;
        Expression  "C2.Value * k8_P";
        VariableReferenceList
            [ C2   :.:C2  -1 ]
            [ CP   :.:CP   1 ];
    }
    
    Process ExpressionFluxProcess( R9 )
    {
        Name  "cdc2k dephosphorylation";
        k9  1000.0;
        Expression  "CP.Value * k9";
        VariableReferenceList
            [ CP   :.:CP  -1 ]
            [ C2   :.:C2   1 ];
    }

### Assignment total ammounts ###

    Process ExpressionAssignmentProcess( YT )
    {
        StepperID  PSV;
        Name  "Assignment rule for 'YT'";
        Expression  "Y.Value + YP.Value + M.Value + pM.Value";
        VariableReferenceList
            [ YT :.:YT  1 ]
            [ Y  :.:Y   0 ]
            [ YP :.:YP  0 ]
            [ M  :.:M   0 ]
            [ pM :.:pM  0 ];
    }
    
    Process ExpressionAssignmentProcess( CT )
    {
        StepperID  PSV;
        Name  "Assignment rule for 'CT'";
        Expression  "C2.Value + CP.Value + M.Value + pM.Value";
        VariableReferenceList
            [ CT :.:CT  1 ]
            [ C2 :.:C2  0 ]
            [ CP :.:CP  0 ]
            [ M  :.:M   0 ]
            [ pM :.:pM  0 ];
    }

}
'''


In [ ]:
setModel( EM, "Tyson1991")
print 't = {}'.format( getCurrentTime() )

## Loggerをつくる

In [ ]:
target_SystemPath_list = ( '/', '/Cell', )
Target_Properties = { 'Variable': ['Value', 'Velocity'], }
Logger_dict = {}

for target_SystemPath in target_SystemPath_list:
    for E_type, Properties in Target_Properties.items():
        for E in getEntityList( E_type, target_SystemPath ):
            for p in Properties:
                FullPN = ':'.join( ( E_type, target_SystemPath, E, p ) )
                Logger_dict[ FullPN ] = createLoggerStub( FullPN )
                Logger_dict[ FullPN ].create()

In [ ]:
step_width = 100.0  # min

run( step_width )
print 't = {} (min)'.format( getCurrentTime() )

## とりあえずシミュレーションしてみる

* まず、オリジナルのモデルを読み込んで 100 分のシミュレーションを実行し、システムパス `/cell` 内のすべての `Variable` をグラフに描いてみます。

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 5, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN.split(':')[3] == 'Value':
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

In [ ]:
for FullPN, d in Data_dict.items():
    if FullPN.split(':')[3] == 'Velocity':
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

## パラメータを変更してシミュレーション結果の変化を観察する

* _k_<sub>_4_</sub>, _k_<sub>_6_</sub> を変更してシミュレーションを実行してみます。
    * このモデルの出典となっている論文の Fig.2 に、さまざまな _k_<sub>_4_</sub>, _k_<sub>_6_</sub> でモデルの振る舞いを検討した結果が示されています。

### _k_<sub>_4_</sub>, _k_<sub>_6_</sub> を変更し、シミュレーションを実行する

* まず、_k_<sub>_4_</sub>, _k_<sub>_6_</sub> が、モデル中のどこにあるのかを確かめます。
* _k_<sub>_4_</sub>, _k_<sub>_6_</sub> は、それぞれ、反応 `R4`, `R6` で用いられているパラメータです。
* R4`, `R6` の FullID は以下の通りです。

```
Process:/Cell:R4
Process:/Cell:R6
```


* `createEntityStub()` で、反応 `R4`, `R6` の `EntityStub` をつくります。

In [ ]:
R4 = createEntityStub( 'Process:/Cell:R4' )
R6 = createEntityStub( 'Process:/Cell:R6' )

* `getPropertyList()` で、反応 `R4`, `R6` の属性を確認し、`k4`, `k6` が含まれていることを確認します。


In [ ]:
R4.getPropertyList()

In [ ]:
R6.getPropertyList()

--------------------------
* Process:/cell:R4:k4 および Process:/cell:R6:k6 を変更します。
* ここでは、_k_<sub>_4_</sub> = 30, _k_<sub>_6_</sub> = 0.1 に変更します。


In [ ]:
R4['k4'] = 30
R6['k6'] = 0.1

* 100 分間のシミュレーションを実行します。
* すでに 100 分のシミュレーションを実行済みなので、現時刻は 200 分となります。


In [ ]:
run( step_width )
print 't = {} (min)'.format( getCurrentTime() )

* 時刻 5 からの Variable の変化をグラフを出力してみます。
  * 時刻 0 近辺は初期値依存の変動が大きいためプロット対象から外します。

In [ ]:
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 5, getCurrentTime(), 0.1 ) )[ :, :2 ]

for FullPN, d in Data_dict.items():
    if FullPN.split(':')[3] == 'Value':
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

* パラメータを変更した _t_ = 100 あたりを境に振動が消えているのが分かります。
* パラメータ変更後の100分間だけをグラフにしてみます。
    * getData() で取得するデータの範囲を **開始**：`getCurrentTime() - 100.0`，**終了**：`getCurrentTime()` とすることで、最後の 100 分間のデータをとってくることができます。
    * 下のセルの最後から２行目に追加された **`plt.xlim()`** は、_x_ 軸の範囲を指定するメソッドです。指定しないと、原点がゼロになります。ここでは、100 ≦ _t_ ≦ 200 の範囲をプロットしたいので、指定を入れます。


In [ ]:
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( getCurrentTime() - 100.0, getCurrentTime(), 0.1 ) )[ :, :2 ]

for FullPN, d in Data_dict.items():
    if FullPN.split(':')[3] == 'Value':
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.xlim( getCurrentTime() - 100.0, getCurrentTime() )
plt.legend( loc = 'upper right' )

* パラメータ変更後も、小さな振動が１回生じ、その後、振動が停止して、_t_ = 140 あたりから、すべての `Variable` に変動がなくなったように見えます。
* これは、初期値依存性と呼ばれる現象の一種で、急激に一部の状況が変化しても、系全体がその変化に追いついて新たな安定状態に達するのに時間がかかるために発生します。  
パラメータを変更して系全体の振る舞いの変化を確認するためには、こうした変動が納まるのに充分な時間のシミュレーションを行ってから、確認しなければなりません。
* ここでは、**パラメータを変更する都度、1000 分のシミュレーションを行う**ことにします。
* 現在の条件でも、追加であと 900 分、シミュレーションを実行し、最後の 100 分のグラフを描くことにします。

In [ ]:
run( 9 * step_width )
print 't = {} (min)'.format( getCurrentTime() )

In [ ]:
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( getCurrentTime() - 100.0, getCurrentTime(), 0.1 ) )[ :, :2 ]

for FullPN, d in Data_dict.items():
    if FullPN.split(':')[3] == 'Value':
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.xlim( getCurrentTime() - 100.0, getCurrentTime() )
plt.legend( loc = 'upper right' )

* 振動が消失し、系全体が変化のない（すべての Variable の変化速度がゼロ）、安定した状態に達しているように見えます。こうした状態を **定常状態（steady state）** といいます。


### _k_<sub>_4_</sub>, _k_<sub>_6_</sub> をいろいろ変更してみるために、関数をつくる

このあと、_k_<sub>_4_</sub>, _k_<sub>_6_</sub> をいろいろ変更して、シミュレーション結果の変化を観察することにします。１組の値を試す毎に、以下のような決まりきったプロセスを繰り返すことになります。

1. `k4`, `k6` の値を書き換える。
2. シミュレーションを実行する（安定するまでのゆとりを考えて 1000 分）。
3. 最後の 100 分間のデータを `Logger` から取得する。
4. データをグラフにプロットする。

これらをまとめて実行する関数（メソッド）を定義することにします。

プログラムを書くときは、原則として、同じことを二度書かないよう心がけるとうまくいくことが多いです。  
同じことを複数回書くと、後にその部分を改変した際に、同じことを書いたすべての箇所を正しく変更しなければならなくなり、この作業をミスを犯さずに繰り返しつづけるのは至難です。人間はミスする生き物です。ミスは起こるものという前提で、ミスをしたときに最小限の労力でミスを発見・訂正できるように、開発プロセスを考えるようにしましょう。これはプログラミングに限りません。

-----
* 関数名は test_k4_and_k6() とし、２つの引数に、設定したい _k_<sub>_4_</sub>, _k_<sub>_6_</sub> の値を順に渡す設計とします。
    * 最終行の plt.title() は、グラフのタイトルを表示させるメソッドです。ここでは、設定した _k_<sub>_4_</sub>, _k_<sub>_6_</sub> の値をタイトルとして表示するようにしてみました。


In [ ]:
def test_k4_and_k6( k4, k6 ):
    R4['k4'] = k4
    R6['k6'] = k6
    
    run( 10 * step_width )
    _t = getCurrentTime()

    for FullPN, Logger in Logger_dict.items():
        Data_dict[ FullPN ] = np.array( Logger.getData( _t - 100.0, _t, 0.1 ) )[ :, :2 ]

    plt.figure()
    for FullPN, d in Data_dict.items():
        if FullPN.split(':')[3] == 'Value':
            plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

    plt.xlim( _t - 100.0, _t )
    plt.legend( loc = 'upper right' )
    plt.title( 'k4 = {},  k6 = {}'.format( k4, k6 ) )

* メソッド test_k4_and_k6 に _k_<sub>_4_</sub>, _k_<sub>_6_</sub> を引数として渡してみます。


In [ ]:
test_k4_and_k6( 30.0, 0.25 )

In [ ]:
test_k4_and_k6( 180.0, 1.0 )

In [ ]:
test_k4_and_k6( 180.0, 2.0 )

In [ ]:
test_k4_and_k6( 500.0, 1.3 )

* _k_<sub>_4_</sub>, _k_<sub>_6_</sub> の組み合わせによって、振動の有無が変化することがわかりました。
* _k_<sub>_4_</sub> = 30, _k_<sub>_6_</sub> = 0.1 および  
_k_<sub>_4_</sub> = 180, _k_<sub>_6_</sub> = 2.0 では振動が消失することが示されましたが、両者の状況は同じでしょうか？各 Variable がどういった値で静止しているかなどを観察して、比較してみてください。

-----
* ここまでの 5100 分のシミュレーション結果全体をグラフに描いてみます。
    * `plt.figure( figsize = (12, 6) )` のように、引数 _`figsize`_` = (横, 縦)` を指定すると、`figure` の大きさを指定できます。
    * 下のグラフでは、凡例がグラフにかぶらないように右に余白をつくるために `plt.xlim()` を使っています。



In [ ]:
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 5, getCurrentTime(), 0.1 ) )[ :, :2 ]

plt.figure( figsize = (12, 6) )
for FullPN, d in Data_dict.items():
    if FullPN.split(':')[3] == 'Value':
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.xlim( 5, getCurrentTime() )
plt.legend( loc = 'upper right' )

* パラメータの変更によって、系の振る舞いが変化している様子を俯瞰することができます。
* ノートブック BioSim1-06-1 に、いくつかのパラメータ値のシミュレーションを実行しグラフを出力する例を示します。

## 参考資料

* [論文PDF](PNAS-1991-Tyson-7328-32.pdf)
* 解説記事：[BioModels データベース Models of the month 2006年10月](http://www.ebi.ac.uk/biomodels-main/static-pages.do?page=ModelMonth%2F2006-10)
* 生物モデルデータベース [BioModels のエントリー](http://www.ebi.ac.uk/biomodels-main/BIOMD0000000005)
* 医学論文データベース [PubMed のエントリー](http://www.ncbi.nlm.nih.gov/pubmed/1831270)